Content-based recommendation systems are a type of recommender system used in information retrieval and e-commerce to provide personalized recommendations to users. These systems recommend items (such as articles, products, movies, music, etc.) to users based on the characteristics or attributes of the items and the preferences of the users.

Here's how content-based recommendation systems work:

Item Representation: Each item in the system is described or represented using a set of relevant attributes or features. For example, in a movie recommendation system, these attributes could include genres, actors, directors, and user ratings.

User Profile: The system creates a user profile that captures the preferences and interests of the user. This profile is typically constructed based on the user's historical interactions, such as items they have rated or liked in the past.

Matching: The system compares the attributes of the items to the user's profile to determine how well they match. This is often done using mathematical similarity measures like cosine similarity or Euclidean distance.

Recommendation: Items that are most similar to the user's profile are recommended to the user. The items that receive the highest similarity scores are typically ranked and presented as recommendations.

Advantages of content-based recommendation systems:

Transparency: Content-based systems are often more transparent than other recommendation approaches like collaborative filtering because they recommend items based on explicit features.

Cold Start Problem: Content-based systems can make recommendations even for new users or items because they rely on item attributes.

Reduced Data Sparsity: They are less affected by data sparsity issues compared to collaborative filtering, which can struggle when there's limited user-item interaction data.

However, content-based recommendation systems also have limitations:

Limited Serendipity: They may have difficulty introducing users to new or unexpected items because recommendations are based on existing user preferences.

Feature Engineering: The quality of recommendations depends heavily on the quality and relevance of the item features, which often requires careful feature engineering.

Over-Specialization: They can sometimes recommend items that are too similar to each other, leading to a lack of diversity in recommendations.

Content-based recommendation systems are commonly used alongside other recommendation techniques, such as collaborative filtering, to improve the quality and diversity of recommendations. Hybrid recommendation systems that combine multiple approaches aim to overcome the limitations of individual methods.

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from tqdm.notebook import tqdm

import joblib
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import nltk

# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:


# if you load in your dataset in google colab
# Load the dataset (replace 'your_dataset.csv' with the actual file name)
#df = pd.read_csv('wiki_movie_plots_deduped.csv')



# if load dataset in gooogle drive
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your CSV file on Google Drive
csv_file_path = '/content/drive/My Drive/Content-based recommendation systems/wiki_movie_plots_deduped.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the head (first 5 rows) of the dataset
df.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [4]:
# List the column names (features) of your dataset
column_names = df.columns.tolist()
print(column_names)


['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast', 'Genre', 'Wiki Page', 'Plot']


In [11]:
df.shape

(34886, 9)

The dataset you've provided appears to be a collection of movie plot summaries. Here's an overview of the data structure and the kind of information it contains:

- Release Year: The year when the movie was released.

- Title: The title of the movie.

- Origin/Ethnicity: This could refer to the country or ethnic origin of the movie.
- Director: The name of the director of the movie.

- Cast: The cast involved in the movie. This field contains missing values in the initial rows.

- Genre: The genre of the movie. Initial entries are labeled as 'unknown'.

- Wiki Page: The Wikipedia page URL for the movie.

- Plot: A brief summary of the movie's plot.

From this initial glance, it seems this dataset can provide insights into the evolution of cinema over time, particularly in terms of genres, directors, and the representation of different cultures or ethnicities in films. It could also be used for tasks like content analysis, studying trends in movie themes and stories, or even for natural language processing tasks like summarization or genre classification based on plot descriptions.



In [ ]:
# Merge all columns into a single text column
df['merged_text'] = df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Display the head of the dataset with the merged_text column
df[['merged_text']].head()


# this cell just want to show me the dataset after merging, after finish cleaning text we have to do this step not now

,merged_text
0,1901 Kansas Saloon Smashers American Unknown u...
1,1901 Love by the Light of the Moon American Un...
2,1901 The Martyred Presidents American Unknown ...
3,"1901 Terrible Teddy, the Grizzly King American..."
4,1902 Jack and the Beanstalk American George S....


Before applying TF-IDF (Term Frequency-Inverse Document Frequency) or any other natural language processing (NLP) techniques, it's common to perform several preprocessing steps on your text data. These **preprocessing steps** can include:



**Text Cleaning**:

- Removing any HTML tags or special characters.

- Lowercasing all text to ensure consistency.
- Removing punctuation marks.
- Handling contractions and special characters (e.g., "don't" to "do not").


**Tokenization:**

- Splitting the text into individual words or tokens.
- Tokenization is usually performed by splitting the text using whitespace or other delimiters.

**Stopword Removal:**

Removing common stopwords (e.g., "the," "and," "is") that do not carry significant meaning.


**Stemming or Lemmatization:**

- Reducing words to their root form Stemming and lemmatization are two common techniques.


**Join Tokens Back into Text**:

- After preprocessing, you may want to join the tokens back into a single text string.

# Data Cleaning of wiki-movie dataset


**Action Items**:

Lower-Case the whole data frame

Director: Removing 'Director:' and 'Cast:'

Director, Cast: Removing '\r\n', '/n' and '/r'

Genre: Replacing '/' with Space

Director, Cast, Genre: Removing 'Uknonwn' and 'Nan'

Director: Separating Directors and Actors names

Director, Cast: Checking if the names are separated with ' and '

Director, Cast: Merging the first names and last names together

Director, Cast: Adding the words of 'Director' and 'Actor' as prefix

Plot: Removing English Stopwords

Doc: Removing special characters

In [6]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import numpy as np

In [7]:

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

In [8]:
# Use a Subset of Dataset, helping to manage memory usage and avoid running out of RAM(overcome RAM memory limitations)
# Randomly sample a fraction of the dataset (e.g., 1.25 %)
# Set a random seed for reproducibility
np.random.seed(42)  # Replace 42 with your chosen seed number
data = df.sample(frac=0.0125)



In [9]:
data.shape

(436, 8)

In [10]:



# Cleaning up the 'Director' and 'Cast' columns
data["Director"] = data["Director"].str.replace("director:", "", regex=False)
data["Director"] = data["Director"].str.replace("cast:", "", regex=False)
data["Director"] = data["Director"].str.replace("\r\n", " ", regex=False)
data["Cast"] = data["Cast"].str.replace("\r\n", " ", regex=False)

# Cleaning up the 'Genre' column
data["Genre"] = data["Genre"].str.replace("/", " ", regex=False)

# Removing 'unknown' and 'nan' from 'Director', 'Cast', 'Genre'
data["Director"] = data["Director"].str.replace("unknown", "", regex=False)
data["Cast"] = data["Cast"].str.replace("unknown", "", regex=False)
data["Genre"] = data["Genre"].str.replace("unknown", "", regex=False)

data["Director"] = data["Director"].str.replace("nan", "", regex=False)
data["Cast"] = data["Cast"].str.replace("nan", "", regex=False)
data["Genre"] = data["Genre"].str.replace("nan", "", regex=False)

# Separating names in 'Director' and 'Cast', merging first and last names, and adding prefixes
data["Director"] = data["Director"].str.replace(" and ", ",", regex=False)
data["Cast"] = data["Cast"].str.replace(" and ", ",", regex=False)

data["Director"] = data["Director"].str.replace(" ", "", regex=False)
data["Cast"] = data["Cast"].str.replace(" ", "", regex=False)

data["Director"] = data["Director"].str.replace(",", " ", regex=False)
data["Cast"] = data["Cast"].str.replace(",", " ", regex=False)

data["Director"] = np.where(data["Director"].str.len() > 0,
                          'director ' + data["Director"],
                          data["Director"])

data["Cast"] = np.where(data["Cast"].str.len() > 0,
                      'actor ' + data["Cast"],
                      data["Cast"])

data["Director"] = data["Director"].str.replace(" ", " director", regex=False)
data["Cast"] = data["Cast"].str.replace(" ", " actor", regex=False)

# Removing English stopwords from 'Plot'
def remove_stopwords(text):
    words = text.split()
    return ' '.join(word for word in words if word not in ENGLISH_STOP_WORDS)

data['Plot'] = data['Plot'].apply(remove_stopwords)

# Show a sample of the cleaned data
data.sample(5)


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
21320,2013,Welcome to the Punch,British,director directorDirector:EranCreevy,actor actorDirector:EranCreevy\nCast:JamesMcAv...,,https://en.wikipedia.org/wiki/Welcome_to_the_P...,"Four men emerge building wearing gas masks, ha..."
23011,1972,Intimate Confessions Of A Chinese Courtesan,Hong Kong,director directorChorYuen,actor actorLilyHo actorBettyPeiTi actorYuehHua,,https://en.wikipedia.org/wiki/Intimate_Confess...,In regarded outrageous taboo-smashing Shaw Bro...
3578,1943,Hi Diddle Diddle,American,director directorAndrewL.Stone,actor actorAdolpheMenjou actorMarthaScott acto...,comedy,https://en.wikipedia.org/wiki/Hi_Diddle_Diddle,Young Janie Prescott married sailor sweetheart...
20542,1990,Nuns on the Run,British,director directorJonathanLynn,actor actorEricIdle actorRobbieColtrane actor,comedy,https://en.wikipedia.org/wiki/Nuns_on_the_Run,"After boss killed bank heist, London gangsters..."
33674,2012,Kaizoku Sentai Gokaiger vs. Space Sheriff Gava...,Japanese,director directorShojiroNakazawa,actor actorRyotaOzawa actorYukiYamada actorMao...,tokusatsu,https://en.wikipedia.org/wiki/Kaizoku_Sentai_G...,The film begins Gokai Galleon chased Super Dim...


# Merging the document

In [11]:
column_weights = {"Release Year": 10,
                  "Title": 1,
                  "Origin/Ethnicity": 5,
                  "Director": 5,
                  "Cast": 1,
                  "Genre": 10,
                  "Plot": 1}

# Initialize a new "doc" column with empty strings
df["doc"] = ""

# Iterate through the columns and concatenate their values based on weights
for col, weight in column_weights.items():
    df["doc"] += df[col].astype(str) + ' '  # Convert to string before concatenation

# Apply the weight to the "doc" column
df["doc"] = df["doc"].str.repeat(weight)

# Optionally, you can strip the trailing space in the "doc" column
df["doc"] = df["doc"].str.strip()

df.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",1901 Kansas Saloon Smashers American Unknown n...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",1901 Love by the Light of the Moon American Un...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",1901 The Martyred Presidents American Unknown ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"1901 Terrible Teddy, the Grizzly King American..."
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,1902 Jack and the Beanstalk American George S....


# Removing Special Characters

In [12]:
df["doc"] = df["doc"].str.replace("[^a-z 0-9]+", "", regex=True)

In [13]:
pd.set_option('display.max_colwidth', None)
df[["doc"]].sample(10)

doc
19730                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1964 he igh right un ritish alph homas irk ogarde eorge hakiris usan trasberg thriller n 1957 uno usan trasberg an merican archaeology student is visiting yprus and staying with the family of her fathers best friend r ndros oseph urst he witnesses an attack by two  gunmen which results in the death of two ritish soldiers but is unable to identify the killers to the local ritish intelligence officer ajor cuire irk ogardeuno then realises that fugitive  eneral kyros regoire slan is hiding in

# Removing Stopwords

In [14]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

In [15]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stops = stopwords.words('english')
print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Text preprocessing function
def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)

    filtered_words = [w for w in tokens if len(w) > 2 if not w in stops]  # Use the 'stops' variable
    stem_words = [stemmer.stem(w) for w in filtered_words]
    lemma_words = [lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)

df['doc_clean'] = df['doc'].map(lambda s: preprocess(s))


In [19]:
df[["doc", 'doc_clean']].sample(10)

,doc,doc_clean
6668,1956 hat ertain eeling merican elvin rank ob ope va arie aint earl ailey eorge anders erry athers comedy beautiful sophisticated ew ork woman who goes by the name unreath enry va arie aint seems to have it all he is not only the private secretary to the wealthy and popular cartoonist arry arkin eorge anders she is also his fianceeut back in ort uron ichigan when she was a girl she was plain old thel ankowski nd she used to be married to another cartoonist the talented but neurotic rancis ignan ob ope who was once an associate of the famed l appne day when arkins syndicate complains that his boyanddog comic strip nips and unty hasnt been as funny as it used to be unreath hatches a scheme arry is leaving on a business trip and she is busy planning their honeymoon so why not hire ignan to ghostwrite the stripignan doesnt want to do it and certainly cant stand the snooty arkin but he needs the money for his psychiatrist who is trying to find out why any setback or stress leads to ignan experiencing a bad case of nausea lot of interesting developments take place in arkins anhattan penthouse while the cartoonist is away ignans strips are humorous and a hit ld feelings begin to stir in unreath having him around arkins housekeeper ussie begins to play matchmaker young orphan orman arrives one day because arkin intends to adopt him ignan is impressed until he discovers that arkins interested only in the publicity not in the child he program erson to erson is coming to do a live interview so arkin wants a cute boy and happy puppy there by his side just like his cartoon figures nips and untyignan is offended e is supposed to find a small dog for arkin but instead brings home one called appy a gigantic hound ignan also draws a cartoon using arkins name portraying nips as a juvenile delinquent nd if that werent enough arkin comes home to find ignan and unreath dressed in matching pajamas each having drunk one too many martiniarkin fires ignan just before the live appearance which ignan proceeds to interrupt by declaring his love for arkins fiancee unreath decides to dump her betrothed and her fancy new name and live happily ever after with ignan orman and appy,1956 hat ertain eel merican elvin rank ope ari aint earl ailey eorg ander erri ather comedi beauti sophist ork woman goe name unreath enri ari aint seem privat secretari wealthi popular cartoonist arri arkin eorg ander also fianceeut back ort uron ichigan girl plain old thel ankowski use marri anoth cartoonist talent neurot ranci ignan ope associ fame appn day arkin syndic complain boyanddog comic strip nip unti hasnt funni use unreath hatch scheme arri leav busi trip busi plan honeymoon hire ignan ghostwrit stripignan doesnt want certainli cant stand snooti arkin need money psychiatrist tri find setback stress lead ignan experienc bad case nausea lot interest develop take place arkin anhattan penthous cartoonist away ignan strip humor hit feel begin stir unreath around arkin housekeep ussi begin play matchmak young orphan orman arriv one day arkin intend adopt ignan impress discov arkin interest public child program erson erson come live interview arkin want cute boy happi puppi side like cartoon figur nip untyignan offend suppos find small dog arkin instead bring home one call appi gigant hound ignan also draw cartoon use arkin name portray nip juvenil delinqu werent enough arkin come home find ignan unreath dress match pajama drunk one mani martiniarkin fire ignan live appear ignan proce interrupt declar love arkin fiance unreath decid dump betroth fanci new name live happili ever ignan orman appi
2455,1938 ky iant merican ew anders oan ontaine ichard ix arry arey drama adventure pon reaching retirement age olonel ornelius tockton arry arey is forced to leave the military accepting a job running the ransorld ir ines chool of eronautics in lendale alifornia tag ahill ichard ix an old friend from the war is the pilot on the commercial airliner taking him to lenda


Lemmatization typically takes more time compared to stemming because it is a more complex and linguistically informed process. Here are some reasons why lemmatization is slower than stemming:

Linguistic Knowledge: Lemmatization relies on linguistic knowledge and rules to find the base or dictionary form (lemma) of a word. This involves understanding the part of speech of the word (e.g., noun, verb, adjective) and applying rules to convert it to its canonical form. Stemming, on the other hand, uses heuristics and simpler rules to remove suffixes and prefixes without considering the grammatical context.

Accuracy: Lemmatization aims to produce accurate base forms of words, which requires more extensive analysis and rule-based transformations. Stemming is a more aggressive approach that may produce less accurate results but is faster because it doesn't consider the context as much.

Lexicon or Dictionary: Lemmatization often requires access to a lexicon or dictionary of words and their lemmas. Stemming usually doesn't rely on a dictionary and operates purely based on pattern matching and rules.

Complexity of the English Language: English has a complex vocabulary with irregular forms and exceptions. Lemmatization algorithms need to account for these irregularities, which can make the process more time-consuming.

Part of Speech: Lemmatization considers the part of speech of a word, which adds an extra layer of complexity. Stemming treats all words as if they were nouns, which simplifies the process.

Contextual Analysis: Lemmatization may require contextual analysis to determine the correct base form based on the surrounding words and the sentence structure. Stemming typically doesn't consider context in this way.

Due to these factors, lemmatization is generally more accurate in producing meaningful base forms of words, but it comes at the cost of increased computational complexity and time. In contrast, stemming is faster but may produce less accurate results and may not always result in valid words. The choice between lemmatization and stemming depends on the specific requirements of the natural language processing task and the trade-off between accuracy and efficiency.

# TF-IDF

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['doc_clean'])
column_names = vectorizer.get_feature_names_out()
df_tf_idf = pd.DataFrame(X.toarray(), columns=column_names)


# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Cosine Similarity
df_cos_sim = pd.DataFrame(cosine_similarity(df_tf_idf, df_tf_idf))
df_cos_sim

# Converting Cosine Similarity Dataframe to Top-K Items

In [ ]:
import warnings

# hide pandas warning messages
warnings.filterwarnings('ignore')

In [ ]:
from tqdm.notebook import tqdm

K = 10
df_top_k = pd.DataFrame(index=df.index)

for col in tqdm(df.index):
    sim_scores = list(enumerate(df_cos_sim[col]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:K+1]
    movie_indices = [i[0] for i in sim_scores]
    df_top_k[col] = movie_indices


In [ ]:
# saving similarity top-k dataframe

# Transpose
df_top_k = df_top_k.T

df_top_k.sample(10)

# saving similarity top-k dataframe
df_top_k.to_parquet("../data/movie_top_k_t.parquet")


# Streamlit Application

'
'
'